# Two-Head MLP Training (TB Outcome + Confidence) — with Priors + Numba Feature Calls

This notebook (v2):
- Uses **generate_tb_labels** only (NO direction label)
- Loads **3 prior feature files**: HMM / Markov / N-gram (parquet)
- Computes **ret_1 / vol_20 / rsi_14** via your `features/numba_feats/*`
- Trains a **true two-head MLP**
  - Head A: TB **outcome** classification {-1,0,+1} (down / no-event / up)
  - Head B: TB **confidence** regression (continuous TB, **hit-only** loss)
- Evaluates event precision/recall, direction-on-event, ranking Precision@Top-K


In [ ]:
# ===== Training / Evaluation Config (Embedded) =====

DATA_SPLIT = {
    "train": ("2020-01-01", "2024-06-30"),
    "val": ("2024-07-01", "2024-12-31"),
    "test": ("2025-01-01", "2025-06-30"),
}

BATCH_SIZE = 1024
EPOCHS = 50
LR = 1e-3
WEIGHT_DECAY = 1e-4
DEVICE = "cuda"

CLIP_VALUE = 5.0

# Two-head loss weight: total = CE(outcome) + LAMBDA_CONF * SmoothL1(conf)
LAMBDA_CONF = 0.5

HIDDEN_DIMS = [128, 128, 64]
DROPOUT = 0.1

BUCKETS = [-1.0, -0.5, -0.2, 0.2, 0.5, 1.0]

# ===== Prior features (exactly the 3 files in your screenshot) =====
PRIOR_FILES = {
    "hmm": "prior_features/hmm_features.parquet",
    "markov": "prior_features/markov_features.parquet",
    "ngram": "prior_features/ngram_features.parquet",
}

# ===== Feature names we will build =====
CONTINUOUS_COLS = ["ret_1", "vol_20", "rsi_14"]

# For priors: auto-select columns by prefix.
NGRAM_PREFIXES = ("ng1_", "ng2_", "ng3_")
MARKOV_PREFIXES = ("mk1_", "mk2_", "mk3_")

# HMM: either "hmm_regime" or "hmm_state" else any "hmm_" prefix columns
HMM_REGIME_CANDIDATES = ("hmm_regime", "hmm_state")


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

from labels.label_generator import generate_tb_labels
from sklearn.metrics import accuracy_score

torch.set_float32_matmul_precision("high")


## 1️⃣ Load Base DataFrame (time anchor)

In [ ]:
from data.loader import load_kline_data

data_file = Path(
    r"C:\Users\user\Desktop\binance-public-data-master\python\data\spot\monthly\klines\BTCUSDT\5m\BTCUSDT-5m-2020_to_2025_10.csv"
)

df_base = load_kline_data(data_file).sort_index()

print("Base df rows:", len(df_base))
print("Base df cols:", len(df_base.columns))
df_base.head()


## 2️⃣ Load Priors (HMM / Markov / N-gram) and Join

In [ ]:
df_hmm = pd.read_parquet(PRIOR_FILES["hmm"]).sort_index()
df_mk = pd.read_parquet(PRIOR_FILES["markov"]).sort_index()
df_ng = pd.read_parquet(PRIOR_FILES["ngram"]).sort_index()

print("hmm:", df_hmm.shape)
print("markov:", df_mk.shape)
print("ngram:", df_ng.shape)

# Strict index-aligned join (no shift)
df = df_base.join([df_hmm, df_mk, df_ng], how="inner")
print("Joined df rows:", len(df))
df.head()


## 3️⃣ Compute Continuous Features via your `features/numba_feats/*`

In [ ]:
from features.numba_feats.price_features import (
    _compute_pct_return,
    _compute_volatility,
)
from features.numba_feats.momentum_features import _compute_rsi

close = df["Close"].to_numpy(dtype=np.float32)

df["ret_1"] = _compute_pct_return(close, 1).astype(np.float32)
df["vol_20"] = _compute_volatility(close, 20).astype(np.float32)
df["rsi_14"] = _compute_rsi(close, 14).astype(np.float32)

CONTINUOUS_COLS = ["ret_1", "vol_20", "rsi_14"]

# Train-only statistics
train_start, train_end = DATA_SPLIT["train"]
train_mask = (df.index >= train_start) & (df.index <= train_end)

train_df = df.loc[train_mask, CONTINUOUS_COLS]

cont_mean = train_df.mean(skipna=True)
cont_std = train_df.std(skipna=True).replace(0, np.nan)

print("Train-only mean:")
print(cont_mean)
print("\nTrain-only std:")
print(cont_std)

# Global z-score + clip [-5, 5]
for c in CONTINUOUS_COLS:
    df[c] = (df[c] - cont_mean[c]) / cont_std[c]
    df[c] = df[c].clip(-5.0, 5.0)

print("Continuous features after train-only zscore + clip:")
print(df[CONTINUOUS_COLS].describe().T)


## 4️⃣ Generate TB Labels (Outcome + Confidence)

- Outcome (classification): `tb.labels ∈ {-1,0,+1}` mapped to `{0,1,2}` for CE
- Confidence (regression): continuous TB value; **loss is hit-only** (outcome != 0)


In [ ]:
tb = generate_tb_labels(df)

# tb.labels is expected to be a pandas Series aligned to df.index
tb_label = tb.labels.astype(np.float32)

# Outcome class for CE: {-1,0,+1} -> {0,1,2}
y_outcome = (tb_label.values.astype(int) + 1).astype(np.int64)

# Confidence / intensity target (continuous TB)
y_conf = tb_label.values.astype(np.float32)

print("TB outcome distribution (0=down,1=no-event,2=up):")
vals, cnts = np.unique(y_outcome, return_counts=True)
print(dict(zip(vals, cnts)))


## 5️⃣ Align & Drop NaNs

- keep only rows where continuous features exist
- TB labels are always aligned (same index), confidence loss will be masked hit-only


In [ ]:
# Align TB series to df index (safety)
tb_label = tb_label.loc[df.index]

mask = pd.Series(True, index=df.index)
for c in CONTINUOUS_COLS:
    mask &= ~df[c].isna()

df = df.loc[mask]
tb_label = tb_label.loc[mask]

y_outcome = (tb_label.values.astype(int) + 1).astype(np.int64)
y_conf = tb_label.values.astype(np.float32)

print("After alignment:", len(df))
df.head()


## 6️⃣ Select Prior Columns for MLP

In [ ]:
# N-gram columns
ng_cols = [c for c in df.columns if c.startswith(NGRAM_PREFIXES)]

# Markov columns
mk_cols = [c for c in df.columns if c.startswith(MARKOV_PREFIXES)]

# HMM columns
hmm_cols = []
for cand in HMM_REGIME_CANDIDATES:
    if cand in df.columns:
        hmm_cols = [cand]
        break
if not hmm_cols:
    hmm_cols = [c for c in df.columns if c.startswith("hmm_")]

print("Selected columns:")
print("  ng_cols:", len(ng_cols))
print("  mk_cols:", len(mk_cols))
print("  hmm_cols:", len(hmm_cols), hmm_cols[:10])

if len(ng_cols) == 0:
    raise ValueError("No ngram columns found. Check ngram_features.parquet column names.")
if len(mk_cols) == 0:
    raise ValueError("No markov columns found. Check markov_features.parquet column names.")
if len(hmm_cols) == 0:
    raise ValueError("No hmm columns found. Check hmm_features.parquet column names.")


## 7️⃣ Time-based Split

In [ ]:
def slice_df(d, start, end):
    return d.loc[(d.index >= start) & (d.index <= end)]

df_train = slice_df(df, *DATA_SPLIT["train"])
df_val = slice_df(df, *DATA_SPLIT["val"])
df_test = slice_df(df, *DATA_SPLIT["test"])

# Targets aligned by index
tb_train_s = tb_label.loc[df_train.index]
tb_val_s = tb_label.loc[df_val.index]
tb_test_s = tb_label.loc[df_test.index]

y_outcome_train = (tb_train_s.values.astype(int) + 1).astype(np.int64)
y_outcome_val = (tb_val_s.values.astype(int) + 1).astype(np.int64)
y_outcome_test = (tb_test_s.values.astype(int) + 1).astype(np.int64)

y_conf_train = tb_train_s.values.astype(np.float32)
y_conf_val = tb_val_s.values.astype(np.float32)
y_conf_test = tb_test_s.values.astype(np.float32)

print("Train rows:", len(df_train), "Val rows:", len(df_val), "Test rows:", len(df_test))


## 8️⃣ Feature-wise Preprocessing (NO global z-score)

- Continuous: clip + zscore using **train stats only**
- N-gram / Markov probabilities: clip to [0,1]
- Other Markov / N-gram scalars: clip + zscore (train-only)
- HMM discrete regime: keep as float32 (no zscore)


In [ ]:
# Identify prob-like columns in priors
prob_suffixes = ("_p_up", "_p_down", "_p_flat", "_maxprob")
ng_prob_cols = [c for c in ng_cols if c.endswith(prob_suffixes)]
mk_prob_cols = [c for c in mk_cols if c.endswith(prob_suffixes)]

# Scalar columns in priors
ng_scalar_cols = [c for c in ng_cols if c not in ng_prob_cols]
mk_scalar_cols = [c for c in mk_cols if c not in mk_prob_cols]

DISCRETE_COLS = hmm_cols

def clip_arr(x, lo, hi):
    return np.clip(x, lo, hi)

# continuous train stats
Xc_train = clip_arr(df_train[CONTINUOUS_COLS].values.astype(np.float32), -CLIP_VALUE, CLIP_VALUE)
c_mean = Xc_train.mean(0, keepdims=True)
c_std = Xc_train.std(0, keepdims=True) + 1e-8

# scalar priors train stats
Xs_train = pd.concat([df_train[ng_scalar_cols], df_train[mk_scalar_cols]], axis=1).values.astype(np.float32)
Xs_train = clip_arr(Xs_train, -CLIP_VALUE, CLIP_VALUE)
s_mean = Xs_train.mean(0, keepdims=True)
s_std = Xs_train.std(0, keepdims=True) + 1e-8

def prep_cont(d):
    x = clip_arr(d[CONTINUOUS_COLS].values.astype(np.float32), -CLIP_VALUE, CLIP_VALUE)
    return (x - c_mean) / c_std

def prep_prob(d, cols):
    return np.clip(d[cols].values.astype(np.float32), 0.0, 1.0)

def prep_scalar(d):
    x = pd.concat([d[ng_scalar_cols], d[mk_scalar_cols]], axis=1).values.astype(np.float32)
    x = clip_arr(x, -CLIP_VALUE, CLIP_VALUE)
    return (x - s_mean) / s_std

def prep_disc(d):
    return d[DISCRETE_COLS].values.astype(np.float32)

def build_X(d):
    return np.hstack([
        prep_cont(d),
        prep_prob(d, ng_prob_cols),
        prep_prob(d, mk_prob_cols),
        prep_scalar(d),
        prep_disc(d),
    ])

X_train = build_X(df_train)
X_val = build_X(df_val)
X_test = build_X(df_test)

print("X_train:", X_train.shape, "X_val:", X_val.shape, "X_test:", X_test.shape)
print("outcome classes train:", np.unique(y_outcome_train, return_counts=True))


## 9️⃣ Dataset / Dataloader (two targets)

In [ ]:
class TwoHeadDS(Dataset):
    def __init__(self, X, y_outcome, y_conf):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y_outcome = torch.tensor(y_outcome, dtype=torch.long)
        self.y_conf = torch.tensor(y_conf, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y_outcome[i], self.y_conf[i]

train_loader = DataLoader(
    TwoHeadDS(X_train, y_outcome_train, y_conf_train),
    batch_size=BATCH_SIZE,
    shuffle=True,
)
val_loader = DataLoader(
    TwoHeadDS(X_val, y_outcome_val, y_conf_val),
    batch_size=BATCH_SIZE,
    shuffle=False,
)

X_test_t = torch.tensor(X_test, dtype=torch.float32)


## 🔟 Two-Head MLP (shared backbone)

In [ ]:
class TwoHeadMLP(nn.Module):
    def __init__(self, d_in, hidden_dims=(128, 128, 64), dropout=0.1):
        super().__init__()
        layers = []
        d = d_in
        for h in hidden_dims:
            layers.append(nn.Linear(d, h))
            layers.append(nn.GELU())
            if dropout and dropout > 0:
                layers.append(nn.Dropout(dropout))
            d = h
        self.backbone = nn.Sequential(*layers)
        self.head_outcome = nn.Linear(d, 3)  # 0=down,1=no-event,2=up
        self.head_conf = nn.Linear(d, 1)     # continuous TB confidence

    def forward(self, x):
        h = self.backbone(x)
        logits_outcome = self.head_outcome(h)
        pred_conf = self.head_conf(h).squeeze(-1)
        return logits_outcome, pred_conf

device = DEVICE if (DEVICE == "cuda" and torch.cuda.is_available()) else "cpu"
model = TwoHeadMLP(X_train.shape[1], hidden_dims=tuple(HIDDEN_DIMS), dropout=DROPOUT).to(device)

opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# Weight no-event class lower; emphasize events
loss_outcome_fn = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 0.4, 1.0], device=device))
loss_conf_fn = nn.SmoothL1Loss()

print("Device:", device)
print("Model params:", sum(p.numel() for p in model.parameters()))


## 1️⃣1️⃣ Training

In [ ]:
for ep in range(1, EPOCHS + 1):
    model.train()
    o_sum = 0.0
    c_sum = 0.0
    n_batches = 0

    for x, y_o, y_c in train_loader:
        x = x.to(device)
        y_o = y_o.to(device)
        y_c = y_c.to(device)

        opt.zero_grad()
        logits_o, pred_conf = model(x)

        # Outcome loss (always)
        loss_o = loss_outcome_fn(logits_o, y_o)

        # Confidence loss (hit-only: outcome != no-event (class 1))
        hit_mask = (y_o != 1)
        if hit_mask.any():
            loss_c = loss_conf_fn(pred_conf[hit_mask], y_c[hit_mask])
        else:
            loss_c = torch.tensor(0.0, device=device)

        loss = loss_o + LAMBDA_CONF * loss_c
        loss.backward()
        opt.step()

        o_sum += float(loss_o.item())
        c_sum += float(loss_c.item())
        n_batches += 1

    if ep % 5 == 0 or ep == 1:
        print(f"Epoch {ep:02d} | outcome_loss={o_sum/n_batches:.4f} | conf_loss={c_sum/n_batches:.4f}")


## 1️⃣2️⃣ Evaluation (Event / Direction-on-Event / Ranking)

In [ ]:
model.eval()
with torch.no_grad():
    logits_o, pred_conf = model(X_test_t.to(device))

pred_outcome = logits_o.argmax(1).cpu().numpy()  # {0,1,2}
true_outcome = y_outcome_test                    # {0,1,2}

print("=== Outcome Head ===")
print("Outcome accuracy:", accuracy_score(true_outcome, pred_outcome))

event_true = (true_outcome != 1)
event_pred = (pred_outcome != 1)

tp = int((event_true & event_pred).sum())
pp = int(event_pred.sum())
ap = int(event_true.sum())

prec = tp / max(pp, 1)
rec = tp / max(ap, 1)

print("Event precision:", prec)
print("Event recall   :", rec)

if ap > 0:
    dir_acc_event = (pred_outcome[event_true] == true_outcome[event_true]).mean()
else:
    dir_acc_event = float("nan")
print("Direction acc | event:", dir_acc_event)

print("\n=== Ranking / Precision@Top-K (by |pred_conf|) ===")
pred_conf_np = pred_conf.cpu().numpy().astype(np.float32)
abs_pred = np.abs(pred_conf_np)
order = np.argsort(-abs_pred)

base_event_rate = event_true.mean()
print("Base event rate:", float(base_event_rate))

for k in [100, 500, 1000]:
    if k > len(order):
        continue
    idx = order[:k]
    topk_event_prec = event_true[idx].mean()
    print(f"Top-{k:4d} | Event Precision={topk_event_prec:.3f}")
